In [1]:
import numpy as np
import pandas as pd
from textrank4zh import TextRank4Keyword, TextRank4Sentence
import jieba
import jieba.analyse
import jieba.posseg as pseg
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [10]:
##read data
file_1 = pd.read_csv("C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\nba_weibo_2019_1028_pm.csv")
file_2 = pd.read_csv("C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\nba_weibo_2019_1029_am.csv")

In [4]:
##data cleansing
#file = file_1
file = pd.concat((file_1, file_2), axis = 0)

# drop重复内容行
file.drop_duplicates(subset='Content', keep='first', inplace = True)
print(file.shape)

# 按关键词drop行
index_list = []
for x in file.Content:
    if ("叶落" in x) | ("买球" in x) | ("滚球" in x):
        index_list.append(file.loc[file['Content'] == x,:].index[0])

file.drop(index_list, axis = 0, inplace= True)
print(file.shape)

#去掉hastag和包含内容
for x in file.Content:
    if "#" in x:
        file.loc[file.loc[file['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\#[^#]*\#','',x)

#去掉[]和包含内容
for x in file.Content:
    if "[" in x:
        file.loc[file.loc[file['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\[[^)]*\]','', x)

#查看#移除情况
index_list_1 = []
for x in file.Content:
    if "#" in x:
        index_list_1.append(file.loc[file['Content'] == x, :].index[0])
        
print(file.loc[index_list_1,"Content"].count())

#查看[]移除情况
index_list_2 = []
for x in file.Content:
    if "[" in x:
        index_list_2.append(file.loc[file['Content'] == x, :].index[0])
        
print(file.loc[index_list_2,"Content"].count())


(1749, 7)
(1451, 7)
15
1


In [5]:
#整合所有内容
weibo_keywords = ''
for content in file.Content:
    weibo_keywords = weibo_keywords + ' ' + '{}'.format(content)
    
file_content = str.encode(weibo_keywords)
#分词
seg_list = jieba.cut(file_content, cut_all=False)
#print(' '.join(seg_list))
#print('-'*100)
# 获取分词和词性
words = pseg.cut(file_content)
#for word, flag in words:
    #print('%s, %s' % (word, flag))
#print('-'*100)

# 通过TF-IDF获取关键词
keywords = jieba.analyse.extract_tags(file_content, topK=30, withWeight=True, allowPOS=('n','nr','ns'))
for item in keywords:
    print(item[0],item[1])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ading\AppData\Local\Temp\jieba.cache
Loading model cost 0.997 seconds.
Prefix dict has been built succesfully.


湖人 0.301850426351278
篮板 0.20124753261455736
热火 0.1736165040417706
老鹰 0.15951820562806526
詹姆斯 0.13189943237061721
头条 0.11557610438453866
独行侠 0.11173872511608479
集锦 0.09813961120642145
球员 0.09374512107542082
球队 0.09149496968794887
篮球 0.08487827804660743
勇士 0.07751669390777743
洛杉矶 0.07433225783775561
主场 0.07429217574865025
文章 0.0701984468396384
特雷 0.06335132404903367
孟菲斯 0.06119025423023691
乔丹 0.06091358910766832
鹈鹕 0.058970546073846636
球星 0.055928841625103906
全场 0.055821132015936206
快船 0.055614240928110986
掘金 0.048982021973032
杜兰特 0.04622004206816293
本场 0.043779641820448884
客场 0.04238640807034394
库兹马 0.042234216032689106
霍华德 0.041872812746051535
伦纳德 0.041170606207605986
浓眉 0.04039746765905549


In [12]:
#!/usr/bin/env python
# coding: utf-8

##file 1 sentiment data cleansing and modeling

from aip import AipNlp

APP_ID = '3028bbf5983244f589876eb27aecfc9e'
API_KEY = '9ad1dcd86ff4446ea71b78346e399835'
SECRET_KEY = '1e9b975a61864bc19437489f7f782b3e'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

# 去掉重复行
file_1.drop_duplicates(subset='Content', keep='first', inplace=True)
print(file_1.shape)

# 去掉两个关键字（来自于营销号）
index_list = []
for x in file_1.Content:
    if ("叶落" in x) | ("买球" in x) | ("滚球" in x):
        index_list.append(file_1.loc[file_1['Content'] == x, :].index[0])

file_1.drop(index_list, axis=0, inplace=True)
print(file_1.shape)

#去掉hastag和包含内容
for x in file_1.Content:
    if "#" in x:
        file_1.loc[file_1.loc[file_1['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\#[^#]*\#','',x)

#去掉[]和包含内容
for x in file_1.Content:
    if "[" in x:
        file_1.loc[file_1.loc[file_1['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\[[^)]*\]','', x)

#查看#移除情况
index_list_1 = []
for x in file_1.Content:
    if "#" in x:
        index_list_1.append(file_1.loc[file_1['Content'] == x, :].index[0])
        
print(file_1.loc[index_list_1,"Content"].count())
print("-"*10)

#查看[]移除情况
index_list_2 = []
for x in file_1.Content:
    if "[" in x:
        index_list_2.append(file_1.loc[file_1['Content'] == x, :].index[0])
        
print(file_1.loc[index_list_2,"Content"].count())
print("-"*10)

#create情感评分函数
def get_score(x):
    x=client.sentimentClassify(str(x).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        #print(x['items'][0]['sentiment'])
        return x['items'][0]['sentiment']
    else:
        # print("error")
        # time.sleep(1.5)
        return get_score(x)

#create正面情感概率函数
def get_pos_prob(x):
    x=client.sentimentClassify(str(x).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        #print(x['items'][0]['positive_prob'])
        return x['items'][0]['positive_prob']
    else:
        # print("error")
        # time.sleep(1.5)
        return get_pos_prob(x)

#create负面情感概率函数
def get_neg_prob(x):
    x=client.sentimentClassify(str(x).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        #print(x['items'][0]['negative_prob'])
        return x['items'][0]['negative_prob']
    else:
        # print("error")
        # time.sleep(1.5)
        return get_neg_prob(x)

#在Dataframe中增加相应的特征列
file_1['sentiment_score'] = file_1['Content'].apply(
    lambda x: get_score(x))

file_1['positive_prob'] = file_1['Content'].apply(
    lambda x: get_pos_prob(x))

file_1['negative_prob'] = file_1['Content'].apply(
    lambda x: get_neg_prob(x))

#save as excel
#file_1.to_excel('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Experiment\\Sentiment_prob_{}.xlsx'.format(file_1.iloc[1][3]))

(1007, 7)
(1007, 7)
3
----------
0
----------


SSLError: HTTPSConnectionPool(host='aip.baidubce.com', port=443): Max retries exceeded with url: /rpc/2.0/nlp/v1/sentiment_classify (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))

In [7]:
#!/usr/bin/env python
# coding: utf-8

##file 2 sentiment data cleansing and modeling

from aip import AipNlp

APP_ID = '3028bbf5983244f589876eb27aecfc9e'
API_KEY = '9ad1dcd86ff4446ea71b78346e399835'
SECRET_KEY = '1e9b975a61864bc19437489f7f782b3e'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)
# 去掉重复行
file_2.drop_duplicates(subset='Content', keep='first', inplace=True)
print(file_2.shape)

# 去掉两个关键字（来自于营销号）
index_list = []
for x in file_2.Content:
    if ("叶落" in x) | ("买球" in x) | ("滚球" in x):
        index_list.append(file_2.loc[file_2['Content'] == x, :].index[0])

file_2.drop(index_list, axis=0, inplace=True)
print(file_2.shape)

#去掉hastag和包含内容
for x in file_2.Content:
    if "#" in x:
        file_2.loc[file_2.loc[file_2['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\#[^#]*\#','',x)

#去掉[]和包含内容
for x in file_2.Content:
    if "[" in x:
        file_2.loc[file_2.loc[file_2['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\[[^)]*\]','', x)

#查看#移除情况
index_list_1 = []
for x in file_2.Content:
    if "#" in x:
        index_list_1.append(file_2.loc[file_2['Content'] == x, :].index[0])
        
print(file_2.loc[index_list_1,"Content"].count())
print("-"*10)

#查看[]移除情况
index_list_2 = []
for x in file_2.Content:
    if "[" in x:
        index_list_2.append(file_2.loc[file_2['Content'] == x, :].index[0])
        
print(file_2.loc[index_list_2,"Content"].count())
print("-"*10)

#create情感评分函数
def get_score(x):
    x=client.sentimentClassify(str(x).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        #print(x['items'][0]['sentiment'])
        return x['items'][0]['sentiment']
    else:
        # print("error")
        # time.sleep(1.5)
        return get_score(x)

#create正面情感概率函数
def get_pos_prob(x):
    x=client.sentimentClassify(str(x).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        #print(x['items'][0]['positive_prob'])
        return x['items'][0]['positive_prob']
    else:
        # print("error")
        # time.sleep(1.5)
        return get_pos_prob(x)

#create负面情感概率函数
def get_neg_prob(x):
    x=client.sentimentClassify(str(x).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        #print(x['items'][0]['negative_prob'])
        return x['items'][0]['negative_prob']
    else:
        # print("error")
        # time.sleep(1.5)
        return get_neg_prob(x)

#在Dataframe中增加相应的特征列
file_2['sentiment_score'] = file_2['Content'].apply(
    lambda x: get_score(x))

file_2['positive_prob'] = file_2['Content'].apply(
    lambda x: get_pos_prob(x))

file_2['negative_prob'] = file_2['Content'].apply(
    lambda x: get_neg_prob(x))

#save as excel
file_2.to_excel('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Experiment\\Sentiment_prob_{}.xlsx'.format(file_2.iloc[1]))

(329, 7)
(329, 7)
4
----------
0
----------


In [36]:
#combine two datasets
file_1 = pd.read_excel('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Experiment\\Sentiment_prob_{}.xlsx'.format(file_1.iloc[1][3]), index_col=0)
file_2 = pd.read_excel('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Experiment\\Sentiment_prob_{}.xlsx'.format(file_2.iloc[1][3]), index_col=0)

file = pd.concat((file_1, file_2), axis = 0)
file = file.reset_index().drop(['index','ID'],axis = 1)

#add one column to specify the date
file['date'] = file.iloc[-1][3]

In [40]:
#perform clustering
data = file[["positive_prob","negative_prob"]]
kmeans = KMeans(n_clusters=3)

# 规范化到 [0,1] 空间
min_max_scaler=preprocessing.MinMaxScaler()
data=min_max_scaler.fit_transform(data)

# kmeans 算法
kmeans.fit(data)
predict_y = kmeans.predict(data)
# 合并聚类结果，插入到原数据中
result = pd.concat((file,pd.DataFrame(predict_y)),axis=1)
result.rename({0:u'聚类结果'},axis=1,inplace=True)
#print(result)
# 将结果导出到CSV文件中
result.to_excel("C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Raw data\\sentiment_cluster_{}.xlsx".format(file.iloc[-1][3]))